# Imports

In [ ]:
import math
import matplotlib
import numpy as np
import pandas as pd
import pickle
import sklearn
import tensorflow as tf
import random
import shutil
import os
from enum import Enum
import imblearn
import time

# Learning Model

In [ ]:
def makeCNNModel(evalMetrics, learningRate, inputSize):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = inputSize))
  model.add(tf.keras.layers.AveragePooling2D((3, 3)))
#   model.add(tf.keras.layers.MaxPooling2D((3, 3))) #Test
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(128, activation = 'relu')) #Try to remove
  model.add(tf.keras.layers.Dense(128, activation = 'relu')) #Try to remove
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer = tf.keras.optimizers.Adam(),
#               optimizer = tf.keras.optimizers.experimental.SGD(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = evalMetrics)
  return model

# Load Data

In [ ]:
allCircuits = ['/data/CSV/'+x+'/' for x in os.listdir('/data/CSV/')]
allPkl = []
for circuit in allCircuits:
  for pkl in os.listdir(circuit):
#     if '.pkl' in pkl and '80' in pkl:
    if '.pkl' in pkl:
      allPkl.append(circuit+pkl)

# split 60(Train)/20(Val)/20(Test)
allDfs = [pd.read_pickle(x, compression='zip') for x in allPkl]
df = pd.concat(allDfs)

In [ ]:
# df = %time pd.read_pickle('/data/AllCircuits.pkl', compression='zip')
df = df.sample(frac=1).reset_index(drop=True) #Shuffle all rows
dfVal = df.sample(frac=0.2)
df = df.drop(dfVal.index)

labels = df.pop(df.columns.values[-1])
valLabels = dfVal.pop(dfVal.columns.values[-1])
trainHyperImages = np.array(df).reshape(len(df),22,33,33)
valHyperImages = np.array(dfVal).reshape(len(dfVal),22,33,33)

# scaler = sklearn.preprocessing.StandardScaler()
# trainHyperImages = scaler.fit_transform(trainHyperImages)
# valHyperImages = scaler.transform(valHyperImages)

# scaler = sklearn.preprocessing.StandardScaler()
# labels = df[33*33*22]
# df[0:33*33*22-1] = scaler.fit_transform(df[0:33*33*22-1]).round(decimals=2)
# df[33*33*22] = labels

# Traning

In [ ]:
batch_size = 32 # is important to ensure that each batch has a decent chance of containing a few positive samples
numEpochs = 100
learningRate = 0.001 #Eh?Predictor=0.05, default=0.001
evalMetrics = [tf.keras.metrics.TruePositives(name='tp'),
               tf.keras.metrics.FalsePositives(name='fp'),
               tf.keras.metrics.TrueNegatives(name='tn'),
               tf.keras.metrics.FalseNegatives(name='fn'),
               tf.keras.metrics.BinaryAccuracy(name='accuracy'),
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc')]

inputSize = (22, 33, 33)

model = makeCNNModel(evalMetrics, learningRate, inputSize)

train_history = model.fit(x=trainHyperImages,
                         y=labels,
                         batch_size=batch_size,
                         validation_data=(valHyperImages, valLabels),
                         epochs=numEpochs)

# Compress All CSVs

In [ ]:
allCircuits = ['/data/CSV/'+x+'/' for x in os.listdir('/data/CSV/')]
allCircuits.sort()
for circuit in allCircuits:
  files = os.listdir(circuit)
  files.sort()
  for csv in files:
    if 'viol' not in csv:
      continue
    if csv[0:csv.find('_viol')]+'.pkl' in files: #Already compressed
      continue
    dfViol = pd.read_csv(circuit+csv, dtype=np.float32, header=None)
    dfSurround = pd.read_csv(circuit+csv[0:csv.find('viol')]+'surround.csv', dtype=np.float32, header=None)
    df = pd.concat([dfViol, dfSurround])
    df.to_pickle(circuit+csv[0:csv.find('_viol')]+'.pkl', compression='zip')